<a href="https://colab.research.google.com/github/sdinesh01/script-storyboard-generator/blob/main/DatasetCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset Creation


1. Sources of the dataset

  I got the dataset from [Kaggle](https://www.kaggle.com/datasets/gufukuro/movie-scripts-corpus). Specifically, I downloaded five of the raw text movie scripts that the dataset authors manually annotated by scene heading, character, dialog, and text. 

  The dataset is licensed for free use with attributions via a Creative Commons license. 

2. Description of the dataset
    
    The original dataset is 2.2 GB, though I only downloaded the individual files that I planned to use. The format of the data was raw text, so I was was left to create the structure of the dataset. 

3. Data models and data structures

    I was unsure how to structure and format the raw text for analysis when I received it. My preliminary thought was to create a dataframe to analyze each movie script separately, then create a merged, final dataframe. I used regular expressions to extract individual scenes from each movie. Then, I created a unique document ID and sceneID for each script. Finally, just as preliminary exploratory data analysis, I created a column for all of the upper case headings in the text string. 

    Columns: scriptID, sceneID, text, upper

    I have included some of my initial code below. I repeated the following process for five total scripts. 

    For next steps, I'd like to find a way to break up each line of text by its individual scene components (character_heading, dialog, text). I'm unsure how structure that secondary DF for analysis. 

In [5]:
import pandas as pd
import numpy as np
import re
import requests

In [17]:
path = '/content/drive/MyDrive/Colab Notebooks/NLP/final/Pans_Labyrinth.txt'
with open(path, 'r') as f:
    data = f.read()

In [18]:
data

'scene_heading: EXT. LABYRINTH - NIGHT\n\ntext: In the foreground, OFELIA - 11 years old, skin white as\nsnow, ruby lips and ebony hair - is sprawled on the ground.\nA thick ribbon of blood runs from her nose.\nBut - the blood is flowing backward into her nostril. Drop\nby drop, the blood leaps up and disappears.\nOfelia\'s pupils dilate-\n\nspeaker_heading: NARRATOR\n\ndialog: Long ago in the Underground Realm...\n\ntext: CAMERA MOVES INTO THE DARK PUPIL-\n\nspeaker_heading: NARRATOR\n\ndialog: ...where there are no lies or pain, there\nlived a Princess who dreamt of the human\nworld.\n\ntext: The sound of muffled wind.\n\nspeaker_heading: NARRATOR\n\ndialog: She dreamt of blue skies, the soft breeze\nand sunshine...\n\ntext: A dark CASTLE takes shape. Here, a tower... there, a\nturret... and lighted windows.\nFinally, the damp, mildewed walls of a water well come into\nfocus.\n\nspeaker_heading: NARRATOR\n\ndialog: One day, eluding her keepers, the\nPrincess escaped.\n\ntext: A woman

In [19]:
line_extractor = r"(START|text|speaker_heading|dialog)([^\f]*?)(?=text|speaker_heading|dialog|END SCENE)"
scene_extractor = r"(scene_heading|text|speaker_heading|dialog)([^\f]*?)(?=scene_heading|\n\n\n\n\n\n)"

In [21]:
by_scene: list = re.findall(scene_extractor, data, flags = re.MULTILINE)
by_scene

[('scene_heading',
  ": EXT. LABYRINTH - NIGHT\n\ntext: In the foreground, OFELIA - 11 years old, skin white as\nsnow, ruby lips and ebony hair - is sprawled on the ground.\nA thick ribbon of blood runs from her nose.\nBut - the blood is flowing backward into her nostril. Drop\nby drop, the blood leaps up and disappears.\nOfelia's pupils dilate-\n\nspeaker_heading: NARRATOR\n\ndialog: Long ago in the Underground Realm...\n\ntext: CAMERA MOVES INTO THE DARK PUPIL-\n\nspeaker_heading: NARRATOR\n\ndialog: ...where there are no lies or pain, there\nlived a Princess who dreamt of the human\nworld.\n\ntext: The sound of muffled wind.\n\nspeaker_heading: NARRATOR\n\ndialog: She dreamt of blue skies, the soft breeze\nand sunshine...\n\ntext: A dark CASTLE takes shape. Here, a tower... there, a\nturret... and lighted windows.\nFinally, the damp, mildewed walls of a water well come into\nfocus.\n\nspeaker_heading: NARRATOR\n\ndialog: One day, eluding her keepers, the\nPrincess escaped.\n\ntext: 

In [22]:
df = pd.DataFrame(by_scene, columns=['headings', 'text'])
df['text'] = 'scene_heading' + df['text'].astype(str) + 'END SCENE'
#print(df['text'][0])

In [23]:
df['sceneID'] = range(len(df))
df['scriptID'] = 0

In [25]:
regex = r"\b[A-Z]+\b"

upper = []
for i in df['text']:
  x: list = re.findall(regex, i, flags = re.MULTILINE)
  upper.append(x)
df['upper'] = upper

In [26]:
df = df[['scriptID','sceneID','text', 'upper']]
df

,scriptID,sceneID,text,upper
0,0,0,scene_heading: EXT. LABYRINTH - NIGHT\n\ntext:...,"[EXT, LABYRINTH, NIGHT, OFELIA, A, NARRATOR, C..."
1,0,1,scene_heading: EXT. BOMBED CITY - DAY\n\ntext:...,"[EXT, BOMBED, CITY, DAY, CAMERA, TRACKS, NARRA..."
2,0,2,scene_heading: INT. MAIN CAR - SAME\n\ntext: L...,"[INT, MAIN, CAR, SAME, OFELIA, CAMERA, CARMEN,..."
3,0,3,scene_heading: EXT. WOODS - SAME\n\ntext: The ...,"[EXT, WOODS, SAME, CARMEN, CARMEN, SERRANO, SE..."
4,0,4,scene_heading: INT. WATER MILL - DAY\n\ntext:...,"[INT, WATER, MILL, DAY, CLOSE, UP, A, THUMB, A..."
5,0,5,scene_heading: EXT. GARDENS - FURTHER - CONTIN...,"[EXT, GARDENS, FURTHER, CONTINUOUS, LABYRINTH,..."
6,0,6,scene_heading: INT. DINING ROOM - NIGHT\n\nspe...,"[INT, DINING, ROOM, NIGHT, VIDAL, VIDAL, VIDAL..."
7,0,7,scene_heading: INT. UPPER BEDROOM - DUSK\n\nte...,"[INT, UPPER, BEDROOM, DUSK, DOCTOR, DOCTOR, CA..."
8,0,8,scene_heading: INT. FOYER/STAIRWAY - NIGHT\n\n...,"[INT, FOYER, STAIRWAY, NIGHT, MERCEDES, PACKAG..."
9,0,9,scene_heading: INT. UPPER BEDROOM - NIGHT\n\nt...,"[INT, UPPER, BEDROOM, NIGHT, CARMEN, CARMEN, O..."
